In [27]:
import pandas as pd
import pyarrow.parquet as pq
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [28]:
df_steam_games = pq.read_table('df_steam_games_limpio_final.parquet').to_pandas()

In [29]:
# Selección de columnas relevantes
data = df_steam_games[['id', 'app_name', 'genres', 'tags', 'specs']]

In [30]:
# Limpieza y preprocesamiento de texto
data['combined_features'] = data['genres'] + ' ' + data['tags'] + ' ' + data['specs']

C:\Users\Blasferp\AppData\Local\Temp\ipykernel_12556\2149226059.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['combined_features'] = data['genres'] + ' ' + data['tags'] + ' ' + data['specs']


In [31]:
data.shape

(2020050, 6)

In [32]:
data = data.groupby('id').first().reset_index()

In [33]:
data = data[data['id']!=0]

In [34]:
data['app_name'].nunique()

30708

In [35]:
data = data.sample(n=3500)

In [36]:
data.shape

(3500, 6)

In [37]:
data.reset_index(drop=True, inplace=True)

In [38]:
data

,id,app_name,genres,tags,specs,combined_features
0,657680,nasway,casual,indie,single player,casual indie single player
1,257710,max gentlemen,casual,free to play,single player,casual free to play single player
2,404750,carrotting brain,action,early access,shared/split screen,action early access shared/split screen
3,512740,flat path,casual,indie,single player,casual indie single player
4,552440,the talos principle vr,none,adventure,single player,none adventure single player
...,...,...,...,...,...,...
3495,660110,depthmera,action,action,single player,action action single player
3496,395880,sky tower,none,indie,single player,none indie single player
3497,657590,grav blazer,action,action,single player,action action single player
3498,457140,oxygen not included,indie,early access,single player,indie early access single player


In [39]:
data.head()

,id,app_name,genres,tags,specs,combined_features
0,657680,nasway,casual,indie,single player,casual indie single player
1,257710,max gentlemen,casual,free to play,single player,casual free to play single player
2,404750,carrotting brain,action,early access,shared/split screen,action early access shared/split screen
3,512740,flat path,casual,indie,single player,casual indie single player
4,552440,the talos principle vr,none,adventure,single player,none adventure single player


In [41]:
data['app_name'].value_counts()

app_name
altitude                                                   2
nasway                                                     1
fsx: steam edition   piper pa 38 tomahawk ii add on        1
rogue singularity                                          1
graviteam tactics: zhalanashkol 1969                       1
                                                          ..
lost in nature                                             1
colorless life                                             1
fantasy grounds   mini dungeon #005: the soularium (5e)    1
army men: toys in space                                    1
disgraced revolutionarys edition dlc                       1
Name: count, Length: 3499, dtype: int64

In [42]:
import pyarrow.parquet as pq
data.to_parquet('df_prueba.parquet')

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
CV = CountVectorizer()
converted_metrix = CV.fit_transform(data['combined_features'])

In [45]:
cosine_similarity = cosine_similarity(converted_metrix)

In [46]:
def get_recommendations(app_name, cosine_sim=cosine_similarity ):
    idx = data[data['app_name'] == app_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 10 juegos similares
    game_indices = [i[0] for i in sim_scores]
    return data['app_name'].iloc[game_indices]



In [47]:
# guardo la matriz

import joblib #pickle es alternativa

joblib.dump(cosine_similarity, 'Matriz.pkl', compress=1)

['Matriz.pkl']

In [48]:
import joblib
with open('Matriz.pkl', 'rb') as file:
        modelo = joblib.load(file)

data = pd.read_parquet('df_prueba.parquet')

In [49]:
data.shape

(3500, 6)

In [50]:
recommendations = get_recommendations('lost in nature')
print(recommendations)

728     dungeons of betrayal
1063         legends of time
1082         majestic trials
1171          lost in nature
1218             calico  co.
Name: app_name, dtype: object
